# 第一部分：在Windows11环境下安装ClickHouse
# Первая часть: Установка ClickHouse в среде Windows 11

### 第一步：启用CPU虚拟化     
### Шаг 1: Включение виртуализации процессора  
打开任务管理器(CTRL+Shift+Esc), 选择性能,查看CPU虚拟化，确认是否已启用  
Откройте диспетчер задач (CTRL+Shift+Esc), выберите "Производительность", проверьте виртуализацию процессора и убедитесь, что она включена.  

### 第二步：启用Hyper-V
### Шаг 2: Включение Hyper-V
在控制面板里的启动和关闭Windows功能里勾选Hyper-V，如果没有这个选项，就使用手动命名Hyper-V.cmd文件的方法进行安装  
В панели управления в разделе "Включение и отключение компонентов Windows" отметьте Hyper-V. Если этой опции нет, выполните установку вручную, создав файл Hyper-V.cmd.  

### 第三步：安装WSL
### Шаг 3: Установка WSL
终端中输入命令`wsl`，启动即可  
Введите команду `wsl` в терминале, чтобы запустить.

### 第四步：安装docker
### Шаг 4: Установка Docker
进入docker官网下载安装包，然后安装即可  
Перейдите на официальный сайт Docker, скачайте установочный файл и выполните установку.  

### 第五步：安装clickhouse  
### Шаг 5: Установка ClickHouse  
#### 1.拉取镜像  
#### 1. Загрузка образа   
打开Windows控制台  
Откройте консоль Windows  
```bash
docker pull yandex/clickhouse-server  
docker pull yandex/clickhouse-client   

#### 2.config配置和users配置映射Windows硬盘目录  
#### 2. Настройка конфигураций и маппинг директорий Windows  

运行临时容器temp-clickhouse-server：
Запустите временный контейнер：  
```bash
docker run --rm -d --name=temp-clickhouse-server yandex/clickhouse-server
```
在D盘根目录创建 D:/docker/conf 文件夹  
Создайте папку D:/docker/conf в корневом каталоге диска D  
```bash
docker cp temp-clickhouse-server:/etc/clickhouse-server/config.xml D:/docker/conf/config.xml  
​docker cp temp-clickhouse-server:/etc/clickhouse-server/users.xml D:/docker/conf/users.xml
```
销毁临时容器：  
Удалите временный контейнер:  
```bash
docker stop temp-clickhouse-server 

#### 3. 测试是否安装成功  
#### 3. Проверка успешности установки

进入docker中的clickhouse bash，运行sql  
Войдите в bash ClickHouse в Docker и выполните SQL-запросы:  
```bash
docker exec -it single-clickhouse-server bash  
clickhouse-client  
show databases;

# 第二部分：实验过程  
# Вторая часть: Экспериментальный процесс

### 第一步：配置ClickHouse集群  
### Шаг 1: Настройка кластера ClickHouse
#### 1.1 创建配置文件目录  
#### 1.1 Создание каталога для конфигурации
在Windows的 D:\docker 目录下，创建以下子目录：  
conf 用于存放配置文件。  
data1、data2、data3 分别作为三个ClickHouse节点的数据存储路径。  
Создайте следующие подкаталоги в директории D:\docker на Windows:  
conf — для хранения конфигурационных файлов.  
data1, data2, data3 — пути для хранения данных трех узлов ClickHouse.  
#### 1.2 编写配置文件 config.xml  
#### 1.2 Создание конфигурационного файла config.xml
添加如下内容：  
Добавьте следующий код:  
```xml
    <remote_servers>
        <test_cluster_two_shards>
            <shard>
                <replica>
                    <host>clickhouse1</host>
                    <port>9000</port>
                </replica>
            </shard>
            <shard>
                <replica>
                    <host>clickhouse2</host>
                    <port>9000</port>
                </replica>
            </shard>
            <shard>
                <replica>
                    <host>clickhouse3</host>
                    <port>9000</port>
                </replica>
            </shard>
        </test_cluster_two_shards>
    </remote_servers>
    <zookeeper>
        <node index="1">
            <host>zoo1</host>
            <port>30000</port>
        </node>
        <node index="2">
            <host>zoo2</host>
            <port>30001</port>
        </node>
        <node index="3">
            <host>zoo3</host>
            <port>30002</port>
        </node>
    </zookeeper>
</clickhouse>

### 第二步：启动ClickHouse集群和ZooKeeper  
### Шаг 2: Запуск кластера ClickHouse и ZooKeeper  

#### 2.1 拉取ZooKeeper镜像并启动  
#### 2.1 Загрузка образа ZooKeeper и запуск  

```bash
docker run -d --name zoo1 -p 30000:2181 zookeeper:3.7  
docker run -d --name zoo2 -p 30001:2181 zookeeper:3.7  
docker run -d --name zoo3 -p 30002:2181 zookeeper:3.7  
```
#### 2.2 启动ClickHouse节点  
#### 2.2 Запуск узлов ClickHouse
启动第一个节点：  
Запустите первый узел:  
```bash
docker run -d --name clickhouse1 -p 9000:9000 -p 8123:8123 ^  
-v D:/docker/conf/config.xml:/etc/clickhouse-server/config.xml ^  
-v D:/docker/data1:/var/lib/clickhouse ^  
yandex/clickhouse-server  
```
启动第二个节点：  
Запустите второй узел:  
```bash
docker run -d --name clickhouse2 -p 9001:9000 -p 8124:8123 ^  
-v D:/docker/conf/config.xml:/etc/clickhouse-server/config.xml ^  
-v D:/docker/data2:/var/lib/clickhouse ^  
yandex/clickhouse-server  
```
启动第三个节点：  
Запустите третий узел:  
```bash
docker run -d --name clickhouse3 -p 9002:9000 -p 8125:8123 ^  
-v D:/docker/conf/config.xml:/etc/clickhouse-server/config.xml ^  
-v D:/docker/data3:/var/lib/clickhouse ^  
yandex/clickhouse-server  

### 第三步：进入ClickHouse环境  
### Шаг 3: Вход в среду ClickHouse  

#### 3.1 连接到任意一个ClickHouse节点的交互式客户端  
#### 3.1 Подключение к интерактивному клиенту любого узла ClickHouse  
```bash
docker exec -it clickhouse1 clickhouse-client  
```
运行结果：  
Результат выполнения:  
```vbnet
ClickHouse client version 22.1.3.7 (official build).  
Connecting to localhost:9000 as user default.  
Connected to ClickHouse server version 22.1.3 revision 54455.  
```
#### 3.2 确认连接到集群  
#### 3.2 Проверка подключения к кластеру  
```sql
SELECT * FROM system.clusters WHERE cluster = 'test_cluster_two_shards';  
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM system.clusters
WHERE cluster = 'test_cluster_two_shards'

Query id: a7e8a21a-8429-4665-920c-ed41b064b84e

┌─cluster─────────────────┬─shard_num─┬─shard_weight─┬─replica_num─┬─host_name─┬─host_address─┬─port─┬─is_local─┬─user────┬─default_database─┬─errors_count─┬─slowdowns_count─┬─estimated_recovery_time─┐
│ test_cluster_two_shards │         1 │            1 │           1 │ 127.0.0.1 │ 127.0.0.1    │ 9000 │        1 │ default │                  │            0 │               0 │                       0 │
│ test_cluster_two_shards │         2 │            1 │           1 │ 127.0.0.2 │ 127.0.0.2    │ 9000 │        0 │ default │                  │            0 │               0 │                       0 │
└─────────────────────────┴───────────┴──────────────┴─────────────┴───────────┴──────────────┴──────┴──────────┴─────────┴──────────────────┴──────────────┴─────────────────┴─────────────────────────┘

2 rows in set. Elapsed: 0.002 sec.

### 第四步：创建分布式表的 SQL 脚本  
### Шаг 4: Создание SQL-скриптов для распределённых таблиц  

#### 4.1 创建本地表：在每个节点上创建一个本地表，用于存储数据，这个表按日期（年月）分区，并且按 date 和 id 排序  
#### 4.1 Создание локальной таблицы: На каждом узле создаётся локальная таблица для хранения данных, которая разделяется по месяцам (YYYYMM) и сортируется по date и id  

```sql
CREATE TABLE default.local_table
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = MergeTree()
PARTITION BY toYYYYMM(date)
ORDER BY (date, id);
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.local_table
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(date)
ORDER BY (date, id)

Query id: f77aeb1a-d3a3-4f9c-9849-4f170540dace

Ok.

0 rows in set. Elapsed: 0.105 sec.

#### 4.2 创建分布式表：将数据写入到每个节点上的本地表。需要指定 shard 和 replica。  
#### 4.2 Создание распределённой таблицы: Данные записываются в локальную таблицу на каждом узле. Необходимо указать shard и replica.  

这里的 Distributed 引擎表示数据将被分布到集群中的各个节点。`rand()` 用来随机分配数据到不同的节点。  
В данном случае движок Distributed указывает, что данные будут распределены между узлами кластера. `rand()` используется для случайного распределения данных по узлам.  

```sql
CREATE TABLE default.distributed_table
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, local_table, rand());
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_table
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, local_table, rand())

Query id: 9393e24e-88c5-4d2a-a1d7-f6730e2452c8

Ok.

0 rows in set. Elapsed: 0.068 sec.

### 第五步：实验 1 - 分区键与分片键的关系  
### Шаг 5: Эксперимент 1 - Взаимосвязь ключей партиционирования и шардинга  

#### 5.1 创建具有相同和不同分片键与分区键的表：  
#### 5.1 Создание таблиц с одинаковыми и разными ключами партиционирования и шардинга:  

为了完成实验，我们需要两种表：  
Для проведения эксперимента нам потребуются два типа таблиц:  

- **相同的分片键与分区键**：例如，按 `date` 分片和分区。  
- **Одинаковые ключи партиционирования и шардинга**: например, шардинг и партиционирование по `date`.  

- **不同的分片键与分区键**：例如，按 `id` 分片，按 `date` 分区。  
- **Разные ключи партиционирования и шардинга**: например, шардинг по `id`, а партиционирование по `date`.

**相同的分片键与分区键**  
**Разные ключи партиционирования и шардинга**
```sql
CREATE TABLE default.same_shard_partition_table
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, local_table, toYYYYMM(date));
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.same_shard_partition_table
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, local_table, toYYYYMM(date))

Query id: 69ea1ceb-5d6a-46c0-98c0-6225a7dbf16a

Ok.

0 rows in set. Elapsed: 0.069 sec.

**不同的分片键与分区键**  
**Разные ключи партиционирования и шардинга**
```sql
CREATE TABLE default.diff_shard_partition_table
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, local_table, id);
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.diff_shard_partition_table
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, local_table, id)

Query id: 65ac6824-e24e-409e-9ab5-d66e9035f0c8

Ok.

0 rows in set. Elapsed: 0.067 sec.

#### 5.2 插入数据： 向这两张表插入大量的随机数据  
#### 5.2 Вставка данных: Вставьте большое количество случайных данных в обе таблицы  

相同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
INSERT INTO default.same_shard_partition_table (date, id, value)
SELECT now(), number, 'value_' || toString(number) FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.same_shard_partition_table (date, id, value) SELECT
    now(),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: 39e1e436-9b6d-4f14-b1af-5bd8433866ff

Ok.

0 rows in set. Elapsed: 0.075 sec. Processed 100.00 thousand rows, 800.00 KB (1.33 million rows/s., 10.66 MB/s.)
```

不同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
INSERT INTO default.diff_shard_partition_table (date, id, value)
SELECT now(), number, 'value_' || toString(number) FROM numbers(100000)
```
运行结果：  
Результат выполнения:   
```sql
INSERT INTO default.diff_shard_partition_table (date, id, value) SELECT
    now(),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: 31d81890-0b93-4c52-8845-00dc4d75279d

Ok.

0 rows in set. Elapsed: 0.178 sec. Processed 100.00 thousand rows, 800.00 KB (562.86 thousand rows/s., 4.50 MB/s.)

#### 5.3 测试读性能  
#### 5.3 Тестирование производительности чтения  

- **顺序读取**：可以测试按时间顺序读取数据的性能  
- **Последовательное чтение**: Тестирование производительности чтения данных в хронологическом порядке  

相同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
SELECT * FROM default.same_shard_partition_table ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.same_shard_partition_table
ORDER BY date ASC
LIMIT 100

Query id: 797192ae-c84c-4922-9bd9-c136f213ae7b

┌────────────────date─┬─id─┬─value────┐
│ 2024-11-22 15:17:52 │  0 │ value_0  │
│ 2024-11-22 15:17:52 │  1 │ value_1  │
 ...
 ...
 ...   
│ 2024-11-22 15:17:52 │ 98 │ value_98 │
│ 2024-11-22 15:17:52 │ 99 │ value_99 │
└─────────────────────┴────┴──────────┘

100 rows in set. Elapsed: 0.035 sec. Processed 24.58 thousand rows, 667.72 KB (704.65 thousand rows/s., 19.14 MB/s.)
```


不同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
SELECT * FROM default.diff_shard_partition_table ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.diff_shard_partition_table
ORDER BY date ASC
LIMIT 100

Query id: b29a5b78-f505-42d4-b696-4d031944dcfa

┌────────────────date─┬─id─┬─value────┐
│ 2024-11-22 15:17:52 │  0 │ value_0  │
│ 2024-11-22 15:17:52 │  1 │ value_1  │
 ...
 ...
 ...   
│ 2024-11-22 15:17:52 │ 98 │ value_98 │
│ 2024-11-22 15:17:52 │ 99 │ value_99 │
└─────────────────────┴────┴──────────┘

100 rows in set. Elapsed: 0.019 sec. Processed 24.58 thousand rows, 667.72 KB (1.27 million rows/s., 34.52 MB/s.)

- **随机读取**：可以测试随机读取记录的性能  
- **Случайное чтение**: Тестирование производительности выборочного чтения записей  

相同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
SELECT * FROM default.same_shard_partition_table WHERE id = 12345 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.same_shard_partition_table
WHERE id = 12345
LIMIT 1

Query id: d2286d06-519c-4830-b7f8-ad3e255abbfa

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 15:18:05 │ 12345 │ value_12345 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.021 sec. Processed 24.58 thousand rows, 484.16 KB (1.19 million rows/s., 23.38 MB/s.)
```
  
  
不同的分片键与分区键  
Разные ключи партиционирования и шардинга  
```sql
SELECT * FROM default.diff_shard_partition_table WHERE id = 12345 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.diff_shard_partition_table
WHERE id = 12345
LIMIT 1

Query id: 9283ee96-9aeb-4f54-af04-85c78e1c2d56

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 15:18:05 │ 12345 │ value_12345 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.022 sec. Processed 24.58 thousand rows, 484.16 KB (1.13 million rows/s., 22.24 MB/s.)

### 第六步：实验 2 - 找出最佳的分片键  
目标是评估以下不同分片策略的性能表现：  
- **按月份分片** (`toYYYYMM(date)`)  
- **按偶数/奇数顺序编号分片** (`id % 2`)  
- **按随机键分片** (`rand()`)  
- **按小时分片** (`toHour(date)`)  
- **按秒分片** (`toSecond(date)`)  

### Шаг 6: Эксперимент 2 - Определение оптимального ключа шардинга  
Цель — оценить производительность различных стратегий шардинга:  
- **Шардинг по месяцам** (`toYYYYMM(date)`)  
- **Шардинг по чётным/нечётным номерам** (`id % 2`)  
- **Шардинг по случайному ключу** (`rand()`)  
- **Шардинг по часам** (`toHour(date)`)  
- **Шардинг по секундам** (`toSecond(date)`)  

#### 6.1 创建本地表：在每个节点上创建一个本地表，用于存储分布式表的数据  
#### 6.1 Создание локальной таблицы: На каждом узле создаётся локальная таблица для хранения данных распределённой таблицы  

此表的分区键为 `toYYYYMM(date)`，主键为 `(date, id)`，保证了按月分区并按时间和编号有序。  
Эта таблица имеет ключ партиционирования `toYYYYMM(date)` и первичный ключ `(date, id)`, что обеспечивает разбиение по месяцам и упорядочение по времени и идентификатору.  

```sql
CREATE TABLE default.date_id_local_table
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = MergeTree()
PARTITION BY toYYYYMM(date)
ORDER BY (date, id);
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.date_id_local_table
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(date)
ORDER BY (date, id)

Query id: 1c7f5f90-a2a4-4d56-9e21-73ef99e6de61

Ok.

0 rows in set. Elapsed: 0.107 sec.

#### 6.2 创建分布式表
针对不同的分片键，分别创建分布式表：  
A. **按月份分片** (`toYYYYMM(date)`)  
B. **按偶数/奇数顺序编号分片** (`id % 2`)  
C. **按随机键分片** (`rand()`)  
D. **按小时分片** (`toHour(date)`)  
E. **按秒分片** (`toSecond(date)`)  

#### 6.2 Создание распределённой таблицы  
Для разных ключей шардинга создаются соответствующие распределённые таблицы:  
A. **Шардинг по месяцам** (`toYYYYMM(date)`)  
B. **Шардинг по чётным/нечётным номерам** (`id % 2`)  
C. **Шардинг по случайному ключу** (`rand()`)  
D. **Шардинг по часам** (`toHour(date)`)  
E. **Шардинг по секундам** (`toSecond(date)`)  

A. 按月份分片：toYYYYMM(date)  
A. Шардинг по месяцам (toYYYYMM(date))  
```sql
CREATE TABLE default.distributed_by_month
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toYYYYMM(date));
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_by_month
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toYYYYMM(date))

Query id: b97331fe-ce53-4d14-968a-505a4d4fefb5

Ok.

0 rows in set. Elapsed: 0.059 sec.

B. 按偶数/奇数编号分片：id % 2  
B. Шардинг по чётным/нечётным номерам (id % 2)  
```sql
CREATE TABLE default.distributed_by_even_odd
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, id % 2);
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_by_even_odd
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, id % 2)

Query id: 8a60d3a1-771b-4654-946c-ff515acf3b11

Ok.

0 rows in set. Elapsed: 0.066 sec.

C. 按随机分片：rand()  
C. Шардинг по случайному ключу (rand())  
```sql
CREATE TABLE default.distributed_by_random
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, rand());
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_by_random
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, rand())

Query id: 9a43fd4b-50eb-45a6-9b3b-0b0d771385a0

Ok.

0 rows in set. Elapsed: 0.064 sec.

D. 按小时编号分片：toHour(date)  
D. Шардинг по часам (toHour(date))  
```sql
CREATE TABLE default.distributed_by_hour
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toHour(date));
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_by_hour
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toHour(date))

Query id: 25921672-4940-4f18-a9de-5e1e08858ea7

Ok.

0 rows in set. Elapsed: 0.103 sec.

E. 按秒编号分片：toSecond(date)  
E. Шардинг по секундам (toSecond(date))  
```sql
CREATE TABLE default.distributed_by_second
(
    date DateTime,
    id UInt32,
    value String
) ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toSecond(date));
```
运行结果：  
Результат выполнения:  
```sql
CREATE TABLE default.distributed_by_second
(
    `date` DateTime,
    `id` UInt32,
    `value` String
)
ENGINE = Distributed(test_cluster_two_shards, default, date_id_local_table, toSecond(date))

Query id: cbbd2f5e-6c20-400d-ac42-b3ddf74d1e66

Ok.

0 rows in set. Elapsed: 0.069 sec.

### 6.2 插入数据  
### 6.2 Вставка данных  

使用 ClickHouse 的 numbers 生成器插入数据（每张表插入相同的数据集）。假设插入 100,000 条随机数据。  
Используйте генератор numbers в ClickHouse для вставки данных (в каждую таблицу вставляется одинаковый набор данных). Предположим, что вставляется 100 000 случайных записей.  


A. 按月份分片：toYYYYMM(date)  
A. Шардинг по месяцам (toYYYYMM(date))  
```sql
INSERT INTO default.distributed_by_month (date, id, value)
SELECT
    now() - INTERVAL number SECOND,
    number,
    concat('value_', toString(number))
FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.distributed_by_month (date, id, value) SELECT
    now() - toIntervalSecond(number),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: b1a86130-7ab0-4028-a914-71db894476af

Ok.

0 rows in set. Elapsed: 0.080 sec. Processed 100.00 thousand rows, 800.00 KB (1.24 million rows/s., 9.94 MB/s.)

B. 按偶数/奇数编号分片：id % 2  
B. Шардинг по чётным/нечётным номерам (id % 2)  
```sql
INSERT INTO default.distributed_by_even_odd (date, id, value)
SELECT
    now() - INTERVAL number SECOND,
    number,
    concat('value_', toString(number))
FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.distributed_by_even_odd (date, id, value) SELECT
    now() - toIntervalSecond(number),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: 109856ae-b9c7-46d2-9805-fc852a4fd311

Ok.

0 rows in set. Elapsed: 0.188 sec. Processed 100.00 thousand rows, 800.00 KB (531.92 thousand rows/s., 4.26 MB/s.)

C. 按随机分片：rand()  
C. Шардинг по случайному ключу (rand())
```sql
INSERT INTO default.distributed_by_random (date, id, value)
SELECT
    now() - INTERVAL number SECOND,
    number,
    concat('value_', toString(number))
FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.distributed_by_random (date, id, value) SELECT
    now() - toIntervalSecond(number),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: 2fe704e6-fa52-46a6-86f2-8d8707f6d98b

Ok.

0 rows in set. Elapsed: 0.187 sec. Processed 100.00 thousand rows, 800.00 KB (534.17 thousand rows/s., 4.27 MB/s.)

D. 按小时编号分片：toHour(date)  
D. Шардинг по часам (toHour(date))
```sql
INSERT INTO default.distributed_by_hour (date, id, value)
SELECT
    now() - INTERVAL number SECOND,
    number,
    concat('value_', toString(number))
FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.distributed_by_hour (date, id, value) SELECT
    now() - toIntervalSecond(number),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: 63c0dff1-846f-4089-8e76-a0638179b299

Ok.

0 rows in set. Elapsed: 0.215 sec. Processed 100.00 thousand rows, 800.00 KB (465.87 thousand rows/s., 3.73 MB/s.)

E. 按秒编号分片：toSecond(date)  
E. Шардинг по секундам (toSecond(date))  
```sql
INSERT INTO default.distributed_by_second (date, id, value)
SELECT
    now() - INTERVAL number SECOND,
    number,
    concat('value_', toString(number))
FROM numbers(100000)
```
运行结果：  
Результат выполнения:  
```sql
INSERT INTO default.distributed_by_second (date, id, value) SELECT
    now() - toIntervalSecond(number),
    number,
    concat('value_', toString(number))
FROM numbers(100000)

Query id: c8226e0c-8f07-4343-bd39-25b23f2fcd56

Ok.

0 rows in set. Elapsed: 0.200 sec. Processed 100.00 thousand rows, 800.00 KB (500.25 thousand rows/s., 4.00 MB/s.)

### 6.4 评估性能  
### 6.4 Оценка производительности  

使用以下查询测试每种分片方式的性能：  
Используйте следующие запросы для тестирования производительности каждой стратегии шардинга:  

#### 6.4.1 顺序读取： 测试按时间顺序读取 100 条记录的性能  
#### 6.4.1 Последовательное чтение: Тестирование производительности чтения 100 записей по порядку врмени  


A. 按月份分片：toYYYYMM(date)  
A. Шардинг по месяцам (toYYYYMM(date))
```sql
SELECT * FROM default.distributed_by_month ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_month
ORDER BY date ASC
LIMIT 100

Query id: 8ef6a70d-646b-4351-9077-83e0664a868f

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
 ...
 ...
 ...   
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
└─────────────────────┴───────┴─────────────┘

100 rows in set. Elapsed: 0.104 sec. Processed 73.73 thousand rows, 2.06 MB (709.56 thousand rows/s., 19.87 MB/s.)

B. 按偶数/奇数编号分片：id % 2  
B. Шардинг по чётным/нечётным номерам (id % 2)
```sql
SELECT * FROM default.distributed_by_even_odd ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_even_odd
ORDER BY date ASC
LIMIT 100

Query id: c7eb3b95-e272-4b8e-b5f0-4f52430c9e47

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
 ...
 ...
 ...   
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
└─────────────────────┴───────┴─────────────┘

100 rows in set. Elapsed: 0.056 sec. Processed 73.73 thousand rows, 2.06 MB (1.33 million rows/s., 37.11 MB/s.)

C. 按随机分片：rand()  
C. Шардинг по случайному ключу (rand())
```sql
SELECT * FROM default.distributed_by_random ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_random
ORDER BY date ASC
LIMIT 100

Query id: 3cf4f6cb-a042-4b11-973c-e00a2482a03a

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
 ...
 ...
 ...   
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
└─────────────────────┴───────┴─────────────┘

100 rows in set. Elapsed: 0.054 sec. Processed 73.73 thousand rows, 2.06 MB (1.36 million rows/s., 37.96 MB/s.)

D. 按小时编号分片：toHour(date)  
D. Шардинг по часам (toHour(date))
```sql
SELECT * FROM default.distributed_by_hour ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_hour
ORDER BY date ASC
LIMIT 100

Query id: f5cd4295-2932-43a7-97e9-c780a145a970

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
 ...
 ...
 ...   
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
└─────────────────────┴───────┴─────────────┘

100 rows in set. Elapsed: 0.065 sec. Processed 73.73 thousand rows, 2.06 MB (1.14 million rows/s., 31.88 MB/s.)

E. 按秒编号分片：toSecond(date)  
E. Шардинг по секундам (toSecond(date))
```sql
SELECT * FROM default.distributed_by_second ORDER BY date LIMIT 100
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_second
ORDER BY date ASC
LIMIT 100

Query id: f9404452-52d2-41e2-8e6c-1dd57f102106

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
│ 2024-11-21 12:17:31 │ 99999 │ value_99999 │
 ...
 ...
 ...   
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
│ 2024-11-21 12:18:20 │ 99950 │ value_99950 │
└─────────────────────┴───────┴─────────────┘

100 rows in set. Elapsed: 0.062 sec. Processed 73.73 thousand rows, 2.06 MB (1.19 million rows/s., 33.27 MB/s.)

#### 6.4.2 随机读取：测试按 id 随机读取 1 条记录的性能  
#### 6.4.2 Случайное чтение: Тестирование производительности случайного чтения 1 записи по id

A. 按月份分片：toYYYYMM(date)  
A. Шардинг по месяцам (toYYYYMM(date))  
```sql
SELECT * FROM default.distributed_by_month WHERE id = 54321 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_month
WHERE id = 54321
LIMIT 1

Query id: 0c1846f0-2229-41ba-bc8f-b800664ecd5b

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 01:01:03 │ 54321 │ value_54321 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.046 sec. Processed 465.54 thousand rows, 2.85 MB (10.18 million rows/s., 62.20 MB/s.)

B. 按偶数/奇数编号分片：id % 2  
B. Шардинг по чётным/нечётным номерам (id % 2)
```sql
SELECT * FROM default.distributed_by_even_odd WHERE id = 54321 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_even_odd
WHERE id = 54321
LIMIT 1

Query id: 385fc55d-ae4b-48f3-a4c5-0b0a42129dbf

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 01:01:03 │ 54321 │ value_54321 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.056 sec. Processed 465.54 thousand rows, 2.85 MB (8.38 million rows/s., 51.20 MB/s.)

C. 按随机分片：rand()  
C. Шардинг по случайному ключу (rand())
```sql
SELECT * FROM default.distributed_by_random WHERE id = 54321 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_random
WHERE id = 54321
LIMIT 1

Query id: 27c54344-09b3-414a-846d-46c0cff43e5d

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 01:02:20 │ 54321 │ value_54321 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.055 sec. Processed 465.54 thousand rows, 2.85 MB (8.43 million rows/s., 51.53 MB/s.)

D. 按小时编号分片：toHour(date)  
D. Шардинг по часам (toHour(date))
```sql
SELECT * FROM default.distributed_by_hour WHERE id = 54321 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_hour
WHERE id = 54321
LIMIT 1

Query id: 57996729-b497-4a5d-9cf9-15b66e574b71

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 01:01:03 │ 54321 │ value_54321 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.061 sec. Processed 465.54 thousand rows, 2.85 MB (7.61 million rows/s., 46.53 MB/s.)

E. 按秒编号分片：toSecond(date)  
E. Шардинг по секундам (toSecond(date))  
```sql
SELECT * FROM default.distributed_by_second WHERE id = 54321 LIMIT 1
```
运行结果：  
Результат выполнения:  
```sql
SELECT *
FROM default.distributed_by_second
WHERE id = 54321
LIMIT 1

Query id: 0f273c8d-1809-4232-a4e7-260de8a89281

┌────────────────date─┬────id─┬─value───────┐
│ 2024-11-22 01:04:06 │ 54321 │ value_54321 │
└─────────────────────┴───────┴─────────────┘

1 rows in set. Elapsed: 0.051 sec. Processed 465.54 thousand rows, 2.85 MB (9.05 million rows/s., 55.33 MB/s.)

#### 6.4.3 聚合查询： 测试分片键在聚合操作中的表现  
#### 6.4.3 Агрегатные запросы: Тестирование производительности ключа шардинга при агрегатных операциях  

A. 按月份分片：toYYYYMM(date)  
A. Шардинг по месяцам (toYYYYMM(date))
```sql
SELECT toYYYYMM(date), count(*) FROM default.distributed_by_month GROUP BY toYYYYMM(date)
```
运行结果：  
Результат выполнения:  
```sql
SELECT
    toYYYYMM(date),
    count(*)
FROM default.distributed_by_month
GROUP BY toYYYYMM(date)

Query id: 9cfabf84-a097-4e7f-8112-0fcc20ec60c8

┌─toYYYYMM(date)─┬─count()─┐
│         202411 │ 1000000 │
└────────────────┴─────────┘

1 rows in set. Elapsed: 0.081 sec. Processed 1.00 million rows, 4.00 MB (12.31 million rows/s., 49.24 MB/s.)

B. 按偶数/奇数编号分片：id % 2  
B. Шардинг по чётным/нечётным номерам (id % 2)
```sql
SELECT id % 2, count(*) FROM default.distributed_by_even_odd GROUP BY id % 2
```
运行结果：  
Результат выполнения:  
```sql
SELECT
    id % 2,
    count(*)
FROM default.distributed_by_even_odd
GROUP BY id % 2

Query id: 19e6cd48-bbb4-42bf-9c64-2142bc41cdc5

┌─modulo(id, 2)─┬─count()─┐
│             0 │  500000 │
│             1 │  500000 │
└───────────────┴─────────┘

2 rows in set. Elapsed: 0.117 sec. Processed 1.00 million rows, 4.00 MB (8.58 million rows/s., 34.31 MB/s.)

D. 按小时编号分片：toHour(date)  
D. Шардинг по часам (toHour(date))
```sql
SELECT toHour(date), count(*) FROM default.distributed_by_hour GROUP BY toHour(date)
```
运行结果：  
Результат выполнения:  
```sql
SELECT
    toHour(date),
    count(*)
FROM default.distributed_by_hour
GROUP BY toHour(date)

Query id: 10d9b366-4dcc-4145-ba99-7fd565856341

┌─toHour(date)─┬─count()─┐
│            0 │   36000 │
│            1 │   36000 │
 ...
 ...
 ...   
│           22 │   36000 │
│           23 │   36000 │
└──────────────┴─────────┘

24 rows in set. Elapsed: 0.058 sec. Processed 1.00 million rows, 4.00 MB (17.23 million rows/s., 68.93 MB/s.)

E. 按秒编号分片：toSecond(date)  
E. Шардинг по секундам (toSecond(date))
```sql
SELECT toSecond(date), count(*) FROM default.distributed_by_second GROUP BY toSecond(date)
```
运行结果：  
Результат выполнения:  
```sql
SELECT
    toSecond(date),
    count(*)
FROM default.distributed_by_second
GROUP BY toSecond(date)

Query id: 2949ea0b-8803-433b-aa5b-6c47c08222f5

┌─toSecond(date)─┬─count()─┐
│              0 │   16668 │
│              1 │   16668 │
 ...
 ...
 ...   
│             58 │   16668 │
│             59 │   16668 │
└────────────────┴─────────┘

60 rows in set. Elapsed: 0.051 sec. Processed 1.00 million rows, 4.00 MB (19.58 million rows/s., 78.30 MB/s.)

# 第三部分：实验结论  
# Третья часть: Заключения эксперимента

## 实验一  
## Эксперимент 1

### 实验结果  
### Результаты эксперимента

#### 顺序读取测试

| 分片策略                | 查询耗时 (秒) | 查询速率 (百万行/秒) | 特点分析                               |
|-------------------------|--------------|---------------------|---------------------------------------|
| 相同分片键与分区键        | 0.035        | 0.70                | 数据按时间分区，查询范围小，主要集中在一个分片，排序操作有一定延迟。   |
| 不同分片键与分区键        | 0.019        | 1.27                | 数据分布在多个分片上，利用多个节点的并行处理能力，整体耗时更短。  时更短。

#### 随机读取测试

| 分片策略                | 查询耗时 (秒) | 查询速率 (百万行/秒) | 特点分析                               |
|-------------------------|--------------|---------------------|---------------------------------------|
| 相同分片键与分区键        | 0.021        | 1.19                | 顺序查询有优化，但随机读取性能一般，索引扫描受限于单一分片。      |
| 不同分片键与分区键        | 0.022        | 1.13                | 数据分布在多个分片，跨分片查询有一定的协调开销，性能略慢。     |


#### Производительность последовательного чтения

| Стратегия шардинга        | Время запроса (сек) | Скорость запроса (миллионы строк/сек) | Характеристики производительности                             |
|--------------------------|--------------------|-------------------------------------|-------------------------------------------------------------|
| Один и тот же ключ шардинга и партиционирования | 0.035              | 0.70                                | Данные разделены по времени, диапазон запроса мал, основной запрос ограничен одним шардом, сортировка добавляет задержку. |
| Разные ключи шардинга и партиционирования      | 0.019              | 1.27                                | Данные распределены по нескольким шардированию, используется параллельная обработка на нескольких узлах. |

#### Производительность случайного чтения

| Стратегия шардинга        | Время запроса (сек) | Скорость запроса (миллионы строк/сек) | Характеристики производительности                             |
|--------------------------|--------------------|-------------------------------------|-------------------------------------------------------------|
| Один и тот же ключ шардинга и партиционирования | 0.021              | 1.19                                | Оптимизация последовательного чтения, но для случайного чтения производительность ограничена одним шардом. |
| Разные ключи шардинга и партиционирования      | 0.022              | 1.13                                | Данные распределены по нескольким шардированию, запросы требуют обращения через несколько шардов. |

### 实验分析
顺序读取性能：  
相同分片键与分区键在单节点上可以高效读取局部连续数据，但受限于单节点处理能力。  
不同分片键与分区键通过分片并行读取提升了性能，但涉及多个分片的数据排序和合并。  

随机读取性能：  
随机读取的差异较小，因为两种情况下索引扫描的复杂度和处理方式接近。  
对于不同分片键与分区键的设计，由于需要跨多个分片查询，协调开销稍大。  

### Экспериментальный анализ  
**Производительность последовательного чтения:**
Тот же ключ шардирования и партицирования позволяет эффективно читать локальные последовательные данные на одном узле, но ограничено возможностями обработки этого узла.  
Разные ключи шардирования и партицирования повышают производительность за счет параллельного чтения, но вовлекают сортировку и слияние данных из разных шардов.  

**Производительность случайного чтения:**
Разница в случайном чтении небольшая, поскольку сложность и методы обработки индексного сканирования схожи для обеих моделей.  
Для разных ключей шардирования и партицирования, поскольку запросы могут охватывать несколько шардов, координация и слияние данных вызывает дополнительные накладные расходы.

### 结论
顺序读取场景：  
如果查询主要集中在时间范围内的小批量数据，相同分片键与分区键的设计更高效。  
如果查询覆盖多个时间范围或涉及大批量数据，不同分片键与分区键的并行处理能力更有优势。  

随机读取场景：  
两种设计差异不大。但如果数据热点集中，不同分片键与分区键通过数据均衡分布能降低热点影响。  

综合推荐：  
如果系统主要是时间序列数据（如日志系统），建议分片键与分区键一致。  
如果随机访问比例较高（如用户行为分析或订单查询），建议分片键与分区键不同，以实现更好的数据分布均衡。

### Выводы
**Сценарий последовательного чтения:**
Если запросы в основном сосредоточены на небольших объемах данных в пределах одного временного диапазона, то проектирование с одинаковыми ключами шардирования и партицирования более эффективно.  
Если запросы охватывают несколько временных диапазонов или требуют обработки больших объемов данных, то параллельная обработка с разными ключами шардирования и партицирования имеет большее преимущество.  

**Сценарий случайного чтения:**
Разница между двумя проектами незначительна. Однако, если данные сосредоточены в "горячих" точках, распределение данных с разными ключами шардирования и партицирования может снизить влияние таких "горячих" точек.  

**Рекомендации:**
Если система в основном обрабатывает временные ряды (например, системы логирования), рекомендуется использовать одинаковые ключи шардирования и партицирования.  
Если система предполагает высокий процент случайных обращений (например, анализ пользовательского поведения или запросы по заказам), рекомендуется использовать разные ключи шардирования и партицирования для достижения лучшего распределения данных.

## 实验二
## Эксперимент 2
### 实验结果
### Результаты эксперимента

1.顺序读取性能
| 分片策略                | 查询耗时 (秒) | 查询速率 (百万行/秒) | 特点分析                               |
|-------------------------|--------------|---------------------|---------------------------------------|
| 按月份分片              | 0.104        | 0.71                | 跨分片排序开销高，性能最差。              |
| 按偶数/奇数编号分片      | 0.056        | 1.33                | 分片均衡，排序开销小，性能优异。            |
| 按随机分片              | 0.054        | 1.36                | 随机分布带来负载均衡，查询性能最佳。        |
| 按小时编号分片          | 0.065        | 1.14                | 分片粒度适中，跨分片排序有一定开销。        |
| 按秒编号分片            | 0.062        | 1.19                | 分片粒度较细，排序开销略高于随机分片。       |

1. Производительность последовательного чтения  

| Стратегия шардинга       | Время запроса (сек) | Скорость запроса (млн строк/сек) | Характеристика                           |
|-------------------------|---------------------|----------------------------------|------------------------------------------|
| Шардинг по месяцам       | 0.104               | 0.71                             | Высокая стоимость сортировки по шартам, худшая производительность. |
| Шардинг по чётным/нечётным номерам | 0.056     | 1.33                             | Хорошее распределение, низкая стоимость сортировки, отличная производительность. |
| Шардинг по случайному ключу | 0.054             | 1.36                             | Равномерное распределение, лучшая производительность. |
| Шардинг по часам         | 0.065               | 1.14                             | Средняя степень гранулярности, сортировка между шардами имеет некоторые издержки. |
| Шардинг по секундам      | 0.062               | 1.19                             | Мелкая гранулярность, сортировка немного дороже случайного шардинга. |


2.随机读取性能  
| 分片策略                | 查询耗时 (秒) | 查询速率 (百万行/秒) | 特点分析                               |
|-------------------------|--------------|---------------------|---------------------------------------|
| 按月份分片              | 0.046        | 10.18               | 查询集中在单个分片，索引扫描开销低，性能最佳。 |
| 按偶数/奇数编号分片      | 0.056        | 8.38                | 查询分布在两个分片，协调开销稍高。          |
| 按随机分片              | 0.055        | 8.43                | 查询分布均匀，索引扫描均衡，性能较优。        |
| 按小时编号分片          | 0.061        | 7.61                | 查询范围分布在多个分片，索引扫描开销高。      |
| 按秒编号分片            | 0.051        | 9.05                | 查询集中范围小，索引扫描快，性能接近最佳。     |

2. Производительность случайного чтения  

| Стратегия шардинга       | Время запроса (сек) | Скорость запроса (млн строк/сек) | Характеристика                           |
|-------------------------|---------------------|----------------------------------|------------------------------------------|
| Шардинг по месяцам       | 0.046               | 10.18                            | Запросы сконцентрированы в одном шарде, низкая стоимость сканирования индекса, наилучшая производительность. |
| Шардинг по чётным/нечётным номерам | 0.056     | 8.38                             | Запросы распределены на два шарда, немного выше стоимость координации. |
| Шардинг по случайному ключу | 0.055             | 8.43                             | Равномерное распределение запросов, сбалансированное сканирование индекса, хорошая производительность. |
| Шардинг по часам         | 0.061               | 7.61                             | Запросы распределены по нескольким шардам, высокая стоимость сканирования индекса. |
| Шардинг по секундам      | 0.051               | 9.05                             | Запросы с небольшим диапазоном, быстрое сканирование индекса, производительность близка к наилучшей. |

3.聚合查询性能  
| 分片策略                | 查询耗时 (秒) | 查询速率 (百万行/秒) | 特点分析                               |
|-------------------------|--------------|---------------------|---------------------------------------|
| 按月份分片              | 0.081        | 12.31               | 分片粒度适中，聚合效率较高。               |
| 按偶数/奇数编号分片      | 0.117        | 8.58                | 跨分片协调聚合开销较大，性能较差。          |
| 按小时编号分片          | 0.058        | 17.23               | 分组数量较少（24小时），协调开销低，性能优秀。 |
| 按秒编号分片            | 0.051        | 19.58               | 分片非常均匀，分组操作分布充分，性能最佳。     |

3. Производительность агрегированных запросов  

| Стратегия шардинга       | Время запроса (сек) | Скорость запроса (млн строк/сек) | Характеристика                           |
|-------------------------|---------------------|----------------------------------|------------------------------------------|
| Шардинг по месяцам       | 0.081               | 12.31                            | Средняя гранулярность шардов, высокая эффективность агрегирования. |
| Шардинг по чётным/нечётным номерам | 0.117     | 8.58                             | Высокие затраты на координацию между шардов, худшая производительность. |
| Шардинг по часам         | 0.058               | 17.23                            | Меньше групп (24 часа), низкие затраты на координацию, отличная производительность. |
| Шардинг по секундам      | 0.051               | 19.58                            | Очень равномерное распределение шардов, хорошая производительность при распределённых группах. |

### 实验分析
### Экспериментальный анализ

1.顺序读取：  
**按随机分片（rand()）和按偶数/奇数编号分片（id % 2）**表现最佳，分片均衡且排序开销较低。  
**按月份分片（toYYYYMM(date)）**由于分片粒度较大，查询需要跨分片排序，性能最差。  

2.随机读取：  
**按月份分片（toYYYYMM(date)）和按秒编号分片（toSecond(date)）**性能最佳，因分片集中，索引扫描效率高。  
**按随机分片（rand()）和按偶数/奇数编号分片（id % 2）**受跨分片协调开销影响，性能稍逊。  

3.聚合查询：  
**按秒编号分片（toSecond(date)）和按小时编号分片（toHour(date)）**表现优秀，分片均匀且分组计算高效。  
**按偶数/奇数编号分片（id % 2）**性能较差，因聚合需要分片间协调。

1. Последовательное чтение:  
**Шардинг по случайному ключу (rand()) и шардинг по чётным/нечётным номерам (id % 2)** показали наилучшую производительность, так как шарды сбалансированы, а стоимость сортировки относительно низкая.  
**Шардинг по месяцам (toYYYYMM(date))** имеет наиболее низкую производительность из-за более крупной гранулярности шардов, что требует сортировки данных между шардов.  

2. Случайное чтение:  
**Шардинг по месяцам (toYYYYMM(date)) и шардинг по секундам (toSecond(date))** показали наилучшую производительность, так как данные сосредоточены в одном шарде, а эффективность сканирования индекса высокая.  
**Шардинг по случайному ключу (rand()) и шардинг по чётным/нечётным номерам (id % 2)** страдают от дополнительных затрат на координацию между шардов, что приводит к несколько худшей производительности.  

3. Агрегированные запросы:  
**Шардинг по секундам (toSecond(date)) и шардинг по часам (toHour(date))** показали отличную производительность, так как шардирование сбалансировано, и операции по группировке выполняются эффективно.  
**Шардинг по чётным/нечётным номерам (id % 2)** показал худшие результаты, так как для агрегирования требуются дополнительные затраты на координацию между шардов.

### 结论

**顺序读取为主的场景：**  
推荐使用按随机分片（rand()）或按偶数/奇数编号分片（id % 2），负载均衡性能最优。  

**随机读取为主的场景：**  
推荐使用按月份分片（toYYYYMM(date)）或按秒编号分片（toSecond(date)），索引扫描效率较高。  

**聚合查询频繁的场景：**  
推荐使用按秒编号分片（toSecond(date)）或按小时编号分片（toHour(date)），分片均匀且聚合开销低。  

**综合性能最佳的分片策略：**  
按秒编号分片（toSecond(date)）兼具随机读取和聚合查询的优秀表现，适合综合性能需求较高的场景。

### Заключение  

**Сценарии с преобладанием последовательного чтения:**  
Рекомендуется использовать шардинг по случайному ключу (rand()) или шардинг по чётным/нечётным номерам (id % 2), так как они обеспечивают наилучшую производительность с точки зрения баланса нагрузки.  

**Сценарии с преобладанием случайного чтения:**  
Рекомендуется использовать шардинг по месяцам (toYYYYMM(date)) или шардинг по секундам (toSecond(date)), так как они обеспечивают высокую эффективность сканирования индекса.  

**Сценарии с частыми агрегированными запросами:**  
Рекомендуется использовать шардинг по секундам (toSecond(date)) или шардинг по часам (toHour(date)), так как они обеспечивают равномерное распределение шардов и низкие затраты на агрегацию.  

**Лучшие стратегии шардинга для общей производительности:**  
Шардинг по секундам (toSecond(date)) показывает отличные результаты как для случайных запросов, так и для агрегированных запросов, и идеально подходит для сценариев с высокими требованиями к общей производительности.